In [18]:
import requests
import datetime
import json

LCD_ENDPOINT_MAIN = "https://fcd.terra.dev/v1/txs/"
CONTRACT = "terra1kc87mu460fwkqte29rquh4hc20m54fxwtsx7gp"
def fetch_txs(LCD_ENDPOIN,CONTRACT,offset=None):
    txs = []
    params = {
        "account": CONTRACT,
        "limit": 100
    }
    if offset is not None:
        params['offset']=offset
    i = 0
    until_day = datetime.date(2021, 5, 15)
    day = datetime.date.today()
    while day >= until_day:
    # while i<1:
        resp = requests.get(LCD_ENDPOIN, params=params)
        # txs = txs + resp.json()['txs']
        n = resp.json()['next']
        params['offset'] = n
        i += 1
        day = datetime.datetime.strptime(
            resp.json()['txs'][0]["timestamp"], "%Y-%m-%dT%H:%M:%SZ").date()
        print(resp.json()['txs'][0]["timestamp"])
        insert_txs(resp.json()['txs'])

    valuable_txs = []
    # for tx in txs:
    #     tx.pop('logs', None)
    #     tx.pop('raw_log', None)
        # if get_uluna(tx,CONTRACT) >0:
        #     valuable_txs.append(tx)
    # return txs
    return []
connection = psycopg2.connect(user="fraud",
        password="12345",
        host="127.0.0.1",
        port="6432")
cursor = connection.cursor()
cursor.execute("select min(fcdtxid) from transactions")
m = cursor.fetchone()
print(m)
cursor.close()
connection.close()
txs = fetch_txs(LCD_ENDPOINT_MAIN,CONTRACT,m[0])
# fd = open("txs-dump-bluna.json", "w")
# fd.write(json.dumps(txs))
# fd.close()

(129658865,)


In [15]:
import psycopg2

def insert_txs(txs):
    connection = psycopg2.connect(user="fraud",
        password="12345",
        host="127.0.0.1",
        port="6432")
    with connection:
        with connection.cursor() as cur:
            for tx in txs:
                cur.execute("insert into transactions(txhash,txdate,fcdtxid,txdata) values(%s,%s,%s,%s)",
                     (tx['txhash'],tx['timestamp'],tx['id'],json.dumps(tx)))

In [14]:
insert_txs(txs)

UniqueViolation: duplicate key value violates unique constraint "transactions_txhash_key"
DETAIL:  Key (txhash)=(5C2B842EBCEF4099EFA4CAD85A128A22886401D14E1C273A752CE1EA27FBA5F4) already exists.


In [11]:
import copy
import json
import psycopg2
CONTRACT = "terra1kc87mu460fwkqte29rquh4hc20m54fxwtsx7gp"
fd = open("txs-dump-bluna-full-messages.json")
txs = json.loads(fd.read())

# create table messages (
#     id bigserial,
#     txhash varchar(64) not null, 
#     date timestamp with time zone not null,
#     message_type varchar(50),
#     sender varchar(100),
#     recepient varchar(100),
#     body jsonb
# );
def save_msgs(msgs):
    connection = psycopg2.connect(user="fraud",
        password="12345",
        host="127.0.0.1",
        port="6432")
    with connection:
        with connection.cursor() as cur:
            for msg in msgs:
                cur.execute("insert into messages(txhash,date,message_type,sender,recepient,body) values(%s,%s,%s,%s,%s,%s)",
                     (msg['hash'],msg['date'],'send',msg['from'],msg['to'],json.dumps(msg)))

def parse_day(date):
    return datetime.datetime.strptime(date, "%Y-%m-%dT%H:%M:%SZ").date()

# message
# sender
# amount
# date

def get_txs_from_db():
    txs = []
    connection = psycopg2.connect(user="fraud",
        password="12345",
        host="127.0.0.1",
        port="6432")
    cursor = connection.cursor("lalala")
    cursor.itersize=1000
    cursor.execute("select txdata from transactions")
    rows = cursor.fetchmany(1000)
    # for row in cursor.fetchall():
    c = 1
    while len(rows)>0:
        print('processing ',c)
        c = c + 1
        for row in rows:
            yield row[0]
        rows = cursor.fetchmany(1000)

        # print(row)
        # for row in rows:
        #     try:
        #         yield row
        #     except:
        #         print(row[0])
        # txs.append(row[0])
    cursor.close()
    connection.close()
    # return txs


def get_send_message_from_logs(tx):
    send_msg = {
        'date':tx["timestamp"],
        'hash':tx['txhash'],
    }
    msgs = []
    type_detect_field = "action"
    if not 'logs' in tx:
        return []
    for log in tx['logs']:
        for e in log['events']:
            if e['type'] != "from_contract":
                continue
            match_contract = False
            match_message_type = False
            for a in e['attributes']:
                if a['key'] == "contract_address":
                    if match_contract and match_message_type:
                        msgs.append(copy.deepcopy(send_msg))
                    send_msg = {
                                    'date':tx["timestamp"],
                                    'hash':tx['txhash'],
                                }
                    match_contract = False
                    match_message_type = False
                    if a['value'] == CONTRACT:
                        match_contract = True
                    continue
                if match_contract and a['key'] == type_detect_field and a['value'] == 'send':
                    match_message_type = True
                    continue
                if match_contract and match_message_type:
                    send_msg[a['key']] = a['value']
            if match_contract and match_message_type:
                msgs.append(copy.deepcopy(send_msg))
    return msgs

messages = []
for tx in get_txs_from_db():
    msgs = get_send_message_from_logs(tx)
    messages = messages + msgs
    # if len(msgs) > 0:
    #     print(msgs)
save_msgs(messages)
print(len(messages))


processing  1
processing  2
processing  3
processing  4
processing  5
processing  6
processing  7
processing  8
processing  9
processing  10
processing  11
processing  12
processing  13
processing  14
processing  15
processing  16
processing  17
processing  18
processing  19
processing  20
processing  21
processing  22
processing  23
processing  24
processing  25
processing  26
processing  27
processing  28
processing  29
processing  30
processing  31
processing  32
processing  33
processing  34
processing  35
processing  36
processing  37
processing  38
processing  39
processing  40
processing  41
processing  42
processing  43
processing  44
processing  45
processing  46
processing  47
processing  48
processing  49
processing  50
processing  51
processing  52
processing  53
processing  54
processing  55
processing  56
processing  57
processing  58
processing  59
processing  60
processing  61
processing  62
processing  63
processing  64
processing  65
processing  66
processing  67
proc

In [29]:
from collections import defaultdict
import base64
import json
import datetime
import plotly
import plotly.graph_objs as go
import plotly.graph_objects as px
from plotly.subplots import make_subplots
CONTRACT = "terra1kc87mu460fwkqte29rquh4hc20m54fxwtsx7gp"
fd = open("txs-dump-bluna.json")
txs = json.loads(fd.read())


def add_date(m, date):
    m['date'] = date
    return m


def find_related_messages(txs, contract):
    msgs = []
    for tx in txs:
        msgs = msgs + [add_date(msg, tx['timestamp']) for msg in tx['tx']
                       ['value']['msg'] if msg['value']['contract'] == contract]
    return msgs


def get_message_type(base64_msg):
    decoded_m = json.loads(base64.b64decode(base64_msg))
    return list(decoded_m.keys())[0]


def get_amount_from_send(base64_msg):
    decoded_m = json.loads(base64.b64decode(base64_msg))
    return decoded_m['send']['amount']


class SendMessage(object):
    def __init__(self, amount, date, sender, sub_message_type):
        self.amount = int(amount)
        self.date = datetime.datetime.strptime(date, "%Y-%m-%dT%H:%M:%SZ").date()
        self.sender = sender
        # one of unbond, swap, deposit_collateral
        # '{"swap":{"belief_price":"1.049714260737302397","max_spread":"0.1"}}'
        # '{"deposit_collateral":{}}'
        # '{"unbond":{}}'
        self.sub_message_type = sub_message_type


send_messages = defaultdict(list)

messages = find_related_messages(txs, CONTRACT)
message_type = defaultdict(int)
send_submessage_type = defaultdict(int)
for m in messages:
    mt = get_message_type(m['value']['execute_msg'])
    message_type[mt] += 1
    if mt == "send":
        smt = get_message_type(json.loads(base64.b64decode(
            m['value']['execute_msg']))['send']['msg'])
        sm = SendMessage(
            get_amount_from_send(m['value']['execute_msg']),
            m['date'],
            m['value']['sender'],
            smt,
        )
        # print(sm.date)
        send_messages[sm.date].append(sm)
        send_submessage_type[smt] += 1


def process_messages(message_type):


    # subtype = filter(lambda s: s.sub_message_type=='message_type',send_messages)
    subtype_days = []
    subtype_days_amount = []
    subtype_days_count = []
    for day,message_list in send_messages.items():
        subtype_days.append(day)
        subtype_days_amount.append(
            sum([m.amount for m in message_list if m.sub_message_type==message_type])
        )
        subtype_days_count.append(
            len([m for m in message_list if m.sub_message_type==message_type])
        )
    return subtype_days,subtype_days_amount,subtype_days_count

subtype_swap_days,subtype_swap_days_amount,subtype_swap_days_count = process_messages('swap')
subtype_unbond_days,subtype_unbond_days_amount,subtype_unbond_days_count = process_messages('unbond')
subtype_deposit_collateral_days,subtype_deposit_collateral_days_amount,subtype_deposit_collateral_days_count = process_messages('deposit_collateral')


In [31]:


for message_type in ['swap','unbond','deposit_collateral']:
    subtype_days,subtype_days_amount,subtype_days_count = process_messages(message_type)
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Scatter(x=subtype_days, y=subtype_days_amount, name=f"Суммы по дням {message_type}"),
                secondary_y=False)
    fig.add_trace(go.Scatter(x=subtype_days, y=subtype_days_count,
                            name=f"Кол-во сообщений {message_type}"), secondary_y=True)
    fig.show()


In [31]:
import requests
import datetime
import json
import plotly
import plotly.graph_objs as go
import plotly.graph_objects as px
from collections import defaultdict
from plotly.subplots import make_subplots
import base64
from statistics import stdev,mean

LCD_ENDPOINT = "https://tequila-fcd.terra.dev/v1/txs/"
CONTRACT = "terra1qzzcc8r04ylr20lwzhlafwynqtmjdnrzlc35jh"

LCD_ENDPOINT = "https://fcd.terra.dev/v1/txs/"
CONTRACT = "terra1mtwph2juhj0rvjz7dy92gvl6xvukaxu8rfv8ts"


def fetch_txs():
    txs = []
    params = {
        "account": CONTRACT,
        "limit": 100
    }
    i = 0
    until_day = datetime.date(2021, 5, 18)
    day = datetime.date.today()
    while day >= until_day:
        resp = requests.get(LCD_ENDPOINT, params=params)
        txs = txs + resp.json()['txs']
        n = resp.json()['next']
        params['offset'] = n
        i += 1
        day = datetime.datetime.strptime(
            resp.json()['txs'][0]["timestamp"], "%Y-%m-%dT%H:%M:%SZ").date()
        print(resp.json()['txs'][0]["timestamp"])

    valuable_txs = []
    for tx in txs:
        tx.pop('logs', None)
        tx.pop('raw_log', None)
        # if get_uluna(tx,CONTRACT) >0:
        #     valuable_txs.append(tx)
    return txs


def get_uluna(tx, contract):
    coins = [(msg['value']['coins'],(base64.b64decode(msg['value']['execute_msg']).decode("utf-8"))) for msg in tx['tx']['value']['msg']
             if 'coins' in msg['value'] and msg['value']['contract'] == contract]
    if len(coins) > 0 and len(coins[0][0]) > 0:
        if coins[0][0][0]['denom'] == 'uluna':
            return (int(coins[0][0][0]['amount']),coins[0][1])
    return (0,"")



def funds_per_day(tx_list):
    funds = defaultdict(lambda: defaultdict(int))
    send_number = defaultdict(int)
    for tx in tx_list:
        day = datetime.datetime.strptime(
            tx["timestamp"], "%Y-%m-%dT%H:%M:%SZ").date()
        uluna,msg = get_uluna(tx, CONTRACT)
        funds[day][msg] = funds[day][msg] + uluna
        send_number[day] += 1
    return funds, send_number


if False:
    txs = fetch_txs()
    fd = open("txs-dump.json", "w")
    fd.write(json.dumps(txs))
    fd.close()
else:
    fd = open("txs-dump.json")
    txs = json.loads(fd.read())


days = []
sums = []
delegation_list = []
funds, delegations = funds_per_day(txs)
funds_by_message = defaultdict(lambda: defaultdict(int))
for k, v in funds.items():
    days.append(k)
    sums.append(sum(list(v.values())))
    delegation_list.append(delegations[k])
    for msg in v.keys():
        funds_by_message[msg][k] = v[msg]

print(stdev(delegation_list))
print(mean(delegation_list))
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=days, y=sums, name="Суммы по дням"),
              secondary_y=False)
fig.add_trace(go.Scatter(x=days, y=delegation_list,
                         name="Кол-во делегаций"), secondary_y=True)
fig.show()

bars = []
for msg,data in funds_by_message.items():
    days = list(data.keys())
    coins = list(data.values())
    bars.append(
        go.Bar(
            name=msg
        ,x=days,y=coins)
    )
plot = px.Figure(data=bars)

plot.update_layout(barmode='stack')

plot.show()


326.6534592747188
788.5416666666666


In [3]:
import psycopg2
import plotly
import plotly.graph_objs as go
import plotly.graph_objects as px
from plotly.subplots import make_subplots
from statistics import stdev, mean
import plotly.figure_factory as ff

connection = psycopg2.connect(user="fraud",
                              password="12345",
                              host="127.0.0.1",
                              port="6432")
cursor = connection.cursor("lalala")
cursor.execute(
    "select date_trunc('day',date) as day ,sum((body->>'amount')::decimal),count(date) from messages group by day;")
days = []
sums = []
amounts = []
for row in cursor.fetchall():
    days.append(row[0])
    sums.append(row[1])
    amounts.append(row[2])

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=days, y=sums, name="Суммы по дням"),
              secondary_y=False)
fig.add_trace(go.Scatter(x=days, y=amounts,
                         name="кол-во send"), secondary_y=True)
fig.show()


def split_range(minValue, maxValue, parts):
    length = (maxValue - minValue)/parts
    splits = []
    for i in range(parts):
        splits.append((i*length, (i+1)*length))
    return splits


ranges = split_range(min(amounts)*0.9, max(amounts)*1.1, 30)
x_axis = list(map(lambda s: (s[1]-s[0])/2+s[0], ranges))
values = [0]*len(x_axis)
for value in amounts:
    for r in enumerate(ranges):
        if value > r[1][0] and value <= r[1][1]:
            values[r[0]] = values[r[0]]+1

s = stdev(amounts)
m = mean(amounts)

figdev = go.Figure()
figdev.add_trace(go.Scatter(x=x_axis, y=values, name="", mode="lines"))
figdev.add_vrect(
    x0=m-s, x1=m+s,
    fillcolor="LightSalmon", opacity=0.5,
    layer="below", line_width=0,
)
figdev.add_vrect(
    x0=m+s, x1=m+s*2,
    fillcolor="LightGreen", opacity=0.5,
    layer="below", line_width=0,
)
figdev.show()

fig = ff.create_distplot([values], ['ff'])
fig.show()
